# automating blog posts to work with jupyter-lite

there is a rare occasion that i write notebooks completely in lite.
most often i write in a conventional environment then need to ammend the content
to work when we are in jupyterlite.

In [1]:
## what do we need to do to make a post work in lite?

* explicitly defined dependencies.

    working on a virtual file system is different than your standard file system.
    normally we don't have to define our environment each time,
    but independent of a real file system - in the browser - we need to install packages each time.
    
* patching shit

    if we use requests then we should used https://github.com/koenvo/pyodide-http

* dealing with `pidgy` and extensions.
    
* some packages won't work in lite so we will throw a warning when we know this fo


    we can infer this information or provide it explicitly in the metadata


## what do we need to do to make a post work in lite?

* explicitly defined dependencies.

    working on a virtual file system is different than your standard file system.
    normally we don't have to define our environment each time,
    but independent of a real file system - in the browser - we need to install packages each time.
    
* patching shit

    if we use requests then we should used https://github.com/koenvo/pyodide-http

* dealing with `pidgy` and extensions.
    
* some packages won't work in lite so we will throw a warning when we know this fo


    we can infer this information or provide it explicitly in the metadata

In [2]:
sometimes i forget imports

sometimes i forget imports

In [3]:
## [`depfinder`](https://github.com/ericdill/depfinder) to find packages in a notebook for python

some of my personal style choices might fail like when i use `__import__`, maybe this is a way to cut dependencies from the list.

## [`depfinder`](https://github.com/ericdill/depfinder) to find packages in a notebook for python

some of my personal style choices might fail like when i use `__import__`, maybe this is a way to cut dependencies from the list.

In [4]:
    from pathlib import Path
    import depfinder, pandas
    __import__("requests_cache").install_cache()

    from pathlib import Path
    import depfinder, pandas
    __import__("requests_cache").install_cache()

In [5]:
    files = pandas.Index(Path.cwd().rglob("*.ipynb"))

    files = pandas.Index(Path.cwd().rglob("*.ipynb"))

In [6]:
    cells = (
        files.to_series().apply(Path.read_text)
        .apply(json.loads).apply(pandas.Series)
        .cells.apply(pandas.Series).stack().apply(pandas.Series)
    )

    cells = (
        files.to_series().apply(Path.read_text)
        .apply(json.loads).apply(pandas.Series)
        .cells.apply(pandas.Series).stack().apply(pandas.Series)
    )

### can haz pidgy?

some of these posts are in `pidgy`, i'll use `%reload_ext pidgy` when that is the situation.
peek in the `cells` to find `pidgy` notebooks.

In [7]:
    (has_pidgy := cells[cells.cell_type.eq("code")].source.apply("".join).groupby(
        pandas.Grouper(level=0)
    ).apply(lambda df: df.str.contains("%[re]*load_ext pidgy").any()))

/home/tbone/Documents/tonyfast/tonyfast/xxii/2022-12-13-github-api-stats.ipynb                                   False
/home/tbone/Documents/tonyfast/tonyfast/xxii/2022-11-12-async-import.ipynb                                       False
/home/tbone/Documents/tonyfast/tonyfast/xxii/2022-12-05-markdown-to-doctest.ipynb                                False
/home/tbone/Documents/tonyfast/tonyfast/xxii/2022-12-16-issue-activity-graphql.ipynb                              True
/home/tbone/Documents/tonyfast/tonyfast/xxii/2022-11-12-pluggy-experiments.ipynb                                 False
                                                                                                                 ...  
/home/tbone/Documents/tonyfast/tonyfast/xxii/.ipynb_checkpoints/2022-12-07--checkpoint.ipynb                     False
/home/tbone/Documents/tonyfast/tonyfast/xxii/.ipynb_checkpoints/2022-11-23-better-dask-shape-checkpoint.ipynb    False
/home/tbone/Documents/tonyfast/tonyfast/xxii/.ip

    (has_pidgy := cells[cells.cell_type.eq("code")].source.apply("".join).groupby(
        pandas.Grouper(level=0)
    ).apply(lambda df: df.str.contains("%[re]*load_ext pidgy").any()))

In [8]:
### get the imports
    def get_import(row: pandas.Series) -> dict:
`get_import` normalizes the cell source code for analysis by `depfinder`.
this method catches those situations or returns the attributes of `depfinder.inspection.ImportFinder`

        source = "".join(row.source)
        if has_pidgy[row.name[0]]:
            source = tangle.render(source)
        try:
            return vars(depfinder.inspection.get_imported_libs(textwrap.dedent(source), row.name[0]))
        except BaseException as e:
            return {}

### get the imports
    def get_import(row: pandas.Series) -> dict:
`get_import` normalizes the cell source code for analysis by `depfinder`.
this method catches those situations or returns the attributes of `depfinder.inspection.ImportFinder`

        source = "".join(row.source)
        if has_pidgy[row.name[0]]:
            source = tangle.render(source)
        try:
            return vars(depfinder.inspection.get_imported_libs(textwrap.dedent(source), row.name[0]))
        except BaseException as e:
            return {}

### evaluate the sources

In [9]:
    import depfinder, pandas, midgy
    __import__("requests_cache").install_cache()

    import depfinder, pandas, midgy
    __import__("requests_cache").install_cache()

In [10]:
    tangle = midgy.python.Python()

    tangle = midgy.python.Python()

In [11]:
    modules = (
        (
            results:=
            cells[cells.cell_type.eq("code")].apply(get_import, axis=1)
            .dropna().apply(functools.partial(pandas.Series, dtype="O"))
        )[results.columns[results.columns.str.endswith("_modules")]]
        .stack().apply(list).apply(functools.partial(pandas.Series, dtype="O")).stack()
    )

    modules = (
        (
            results:=
            cells[cells.cell_type.eq("code")].apply(get_import, axis=1)
            .dropna().apply(functools.partial(pandas.Series, dtype="O"))
        )[results.columns[results.columns.str.endswith("_modules")]]
        .stack().apply(list).apply(functools.partial(pandas.Series, dtype="O")).stack()
    )

a snapshot of the modules import within the content

In [12]:
    HTML(modules.value_counts().to_frame().T.to_html())

,pandas,pathlib,toolz,IPython,requests,midgy,json,dataclasses,functools,pidgy,ipywidgets,info,dask,shlex,sys,playwright,jsonref,pluggy,importnb,traitlets,uritemplate,depfinder,types,__,typing,re,ast,gc,io,nbformat,urllib.parse,jsonpointer,numpy,pyld,orjson,linecache,operator,doit,requests_cache,poser,matplotlib,rich,importlib,inspect,anyio,mpl_toolkits,__12_09_pyproject_analysis,icalendar,unittest.mock,tomli,doctest,__static_notebook_tags,hvplot,unittest,textwrap,__11_12_async_import,pytest,a_little_markdown_program,_022_10_21_markdown_future,arrow,nbconvert_html5,bs4,__better_dask_shape,asyncio,dis,traceback,tqdm,ibis,sympy,__pycache__,micropip,duckdb,pyarrow,nest_asyncio,collections,abc,typer
0,35,20,11,10,10,10,9,7,6,6,6,6,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,3,3,3,3,3,3,3,3,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


    HTML(modules.value_counts().to_frame().T.to_html())

## todo

* inject the imports back into the notebooks. where though?
* find magics